In [1]:
import json
import requests
import requests_cache
import pandas as pd
import pprint
import numpy as np
import isodbtools
from scipy.interpolate import *

%matplotlib inline
import matplotlib.pyplot as plt

requests_cache.install_cache('api_map_cache')

In [2]:
# Force use of ipv4
import socket
import requests.packages.urllib3.util.connection as urllib3_cn
 
def allowed_gai_family():
    family = socket.AF_INET    # force IPv4
    return family
 
urllib3_cn.allowed_gai_family = allowed_gai_family

# Download Helpful Data Files

In [3]:
# Build a Library of Adsorptives
#   NOTE: this will accelerate name <-> InChIKey mapping
URL = 'https://adsorption.nist.gov/isodb/api/gases.json'
adsorptives = requests.get(URL).json()

# Build a Library of Adsorbents
URL = 'https://adsorption.nist.gov/matdb/api/materials.json'
adsorbents = requests.get(URL).json()

# Download the (abbreviated) Isotherm Library
URL = 'https://adsorption.nist.gov/isodb/api/isotherms.json'
isotherms = requests.get(URL).json()

# Define Search Criteria

In [4]:
# Adsorptive Species
adsorptive = 'CO2'
adsorptive = requests.get('https://adsorption.nist.gov/isodb/api/gas/'+adsorptive+'.json').json()
pprint.pprint(adsorptive)
target_InChIKey = adsorptive['InChIKey']

# Temperature Range
temperature_min = 273 #K
temperature_max = 323 #K

{'InChICode': 'InChI=1S/CO2/c2-1-3',
 'InChIKey': 'CURLTUGMZLYLDI-UHFFFAOYSA-N',
 'formula': 'CO2',
 'name': 'Carbon Dioxide',
 'synonyms': ['Carbon dioxide', 'Carbon oxide', 'CO2']}


# Filter Isotherms Library against first-level criteria

In [5]:
candidate_isotherms = []

for index,isotherm in enumerate(isotherms):
    # Select single-component isotherms with desired adsorptive
    if len(isotherm['adsorbates']) == 1 and isotherm['adsorbates'][0]['InChIKey'] == target_InChIKey:
        if temperature_min <= isotherm['temperature'] <= temperature_max:
            #print(isotherm['filename'], isotherm['temperature'])
            candidate_isotherms.append(isotherm['filename'])
    #if index == 1000: break

# Filter Isotherms against second-level criteria

In [6]:
Rg = 8.314462618  #J/mol K  = kPa l / mol K  CODATA 2018 [exact]

p_atm = 1.01325 #bar
T_atm = 298 #K
y_CO2 = 400. / 1.e6  # using 400 ppm CO2
p_CO2_atm = y_CO2 * p_atm  # partial pressure of CO2 at atmospheric conditions

In [7]:
def spline_interpolate_loading(pressure,adsorption,p_interp):
    # Add the origin if it's not already there
    if min(pressure) != 0.:
        pressure = [0.] + pressure
        adsorption = [0.] + adsorption
    if min(pressure) < 0.:
        raise Exception('ERROR: negative pressure encountered')
    # Use a spline to interpolate the isotherm
    ads_spline = splrep(pressure,adsorption)
    ads_interp = splev(p_interp,ads_spline)
    return ads_interp

def linear_interpolate_loading(pressure,adsorption,p_interp):
    # Add the origin if it's not already there
    if min(pressure) != 0.:
        pressure = [0.] + pressure
        adsorption = [0.] + adsorption
    if min(pressure) < 0.:
        raise Exception('ERROR: negative pressure encountered')
    # Linearly interpolate the isotherm
    ads_linear = interp1d(pressure, adsorption)
    ads_interp = ads_linear(p_interp)
    return ads_interp    

In [44]:
filtered_isotherms_dict = []
STP_molar_V = 24.4  # cm^3/mmol @ STP conditions

for index,filename in enumerate(candidate_isotherms):
    URL = 'https://adsorption.nist.gov/isodb/api/isotherm/'+filename+'.json&k=dontrackmeplease'
    isotherm_full = requests.get(URL).json()
    #pprint.pprint(isotherm_full)
    T_iso = isotherm_full["temperature"]
    if 260. < T_iso < 330:
        pass
    else:
        continue
    p_CO2_iso = p_CO2_atm * T_iso / T_atm  # equivalent pressure for isotherm temperature
    q_units = isotherm_full['adsorptionUnits']
    #print(p_CO2_iso, p_CO2_atm)
    
    p = [x['pressure'] for x in isotherm_full['isotherm_data']]
    q = [x['species_data'][0]['adsorption'] for x in isotherm_full['isotherm_data']]
    try:
        #loading_estimate = spline_interpolate_loading(p,q,p_CO2_iso)
        loading_estimate = linear_interpolate_loading(p,q,p_CO2_iso)
        #print( isotherm_full['adsorbent']['name'], isotherm_full['temperature'], 'K',
        #       p_CO2_iso, 'bar', loading_estimate, q_units)
        # Convert Loading Units when possible
        if isotherm_full['adsorptionUnits'] == 'cm3(STP)/g':
            # Volumetric loading (STP) to molar loading
            # divide by IG STP molar volume in cm3/mmol
            loading_estimate *= 1./STP_molar_V
            isotherm_full['adsorptionUnits'] = 'mmol/g'
        elif isotherm_full['adsorptionUnits'] == 'mol/g':
            # molar units conversion
            # multiply by 1000 mmol/mol
            loading_estimate *= 1000.
            isotherm_full['adsorptionUnits'] = 'mmol/g'
        elif isotherm_full['adsorptionUnits'] == 'mmol/kg':
            # molar units conversion
            # divide by 1000 g/kg
            loading_estimate *= (1./1000.)
            isotherm_full['adsorptionUnits'] = 'mmol/g'
        
        if loading_estimate > 0:
            filtered_isotherms_dict.append({'DOI' : isotherm_full['DOI'],
                                            'name': isotherm_full['adsorbent']['name'],
                                            'hashkey': isotherm_full['adsorbent']['hashkey'],
                                            'T': isotherm_full['temperature'],
                                            'p': p_CO2_iso,
                                            'q': loading_estimate,
                                            'q_units': isotherm_full['adsorptionUnits'],
                                            'filename': isotherm_full['filename']
                                           })
        else:
            print('negative adsorption encountered', filename)
        
    except:
        print('negative pressure encountered', filename)
    
    #if index == 2000: break
    if (index+1)%50 == 0: print('finished ', index)

# Turn the nasty results in to a nice Pandas Dataframe
filtered_isotherms = pd.DataFrame(
    list(zip(
            [x['DOI'] for x in filtered_isotherms_dict], 
            [x['filename'] for x in filtered_isotherms_dict],
            [x['name'] for x in filtered_isotherms_dict],
            [x['T'] for x in filtered_isotherms_dict],
            [x['p'] for x in filtered_isotherms_dict],
            [x['q'] for x in filtered_isotherms_dict],
            [x['q_units'] for x in filtered_isotherms_dict],
            )),
    columns =['DOI',
              'filename',
              'Material Name',
              'Temperature (K)',
              'Equivalent CO2 Pressure (bar)',
              'Loading Estimate',
              'Loading Units'
             ]
)

#filtered_isotherms

filtered_isotherms.to_csv('co2_loading.csv')

negative adsorption encountered 10.1002adfm.201502069.Isotherm7
negative adsorption encountered 10.1002adma.201403827.Isotherm9
finished  49
negative adsorption encountered 10.1002Aic.13970.isotherm3
negative adsorption encountered 10.1002Aic.13970.isotherm4
negative adsorption encountered 10.1002Aic.13970.isotherm45
negative pressure encountered 10.1002Aic.14467.Isotherm14
negative pressure encountered 10.1002Aic.14467.Isotherm17
negative adsorption encountered 10.1002Aic.14467.Isotherm21
negative pressure encountered 10.1002Aic.14467.Isotherm5
finished  99
negative adsorption encountered 10.1002Aic.14467.Isotherm9
negative adsorption encountered 10.1002anie.201105966.Isotherm9
finished  149
finished  199
finished  249
finished  299
finished  349
finished  399
finished  449
finished  499
negative adsorption encountered 10.1007s10450-018-9958-x.Lab08
negative adsorption encountered 10.1007s10450-018-9958-x.Lab11
finished  549
finished  599
negative adsorption encountered 10.1007s109340

negative adsorption encountered 10.1039C3cp55416c.Isotherm10
finished  4899
finished  4949
finished  4999
negative pressure encountered 10.1039C4cc00334a.Isotherm3
finished  5049
negative pressure encountered 10.1039C4cc06697a.Isotherm1
negative pressure encountered 10.1039C4ce01237b.Isotherm6
finished  5099
negative pressure encountered 10.1039C4ee00143e.Isotherm1
negative pressure encountered 10.1039C4ee00143e.Isotherm2
finished  5149
finished  5199
negative adsorption encountered 10.1039c4ta03992k.Isotherm23
negative adsorption encountered 10.1039c4ta03992k.Isotherm24
negative adsorption encountered 10.1039c4ta03992k.Isotherm25
negative adsorption encountered 10.1039c4ta03992k.Isotherm26
negative adsorption encountered 10.1039c4ta03992k.Isotherm27
negative adsorption encountered 10.1039c4ta03992k.Isotherm28
negative adsorption encountered 10.1039c4ta04770b.Isotherm17
finished  5249
finished  5299
negative adsorption encountered 10.1039c5cc04808g.Isotherm8
finished  5349
finished  53

In [34]:
units_types = [x['q_units'] for x in filtered_isotherms_dict]
print(set(units_types))

{'', 'mmol/m2', 'mol/mol', 'scf/ton', 'volume/volume', 'kmol/m3', 'mol/m2', 'mmol/cm3', 'mg/g', 'molecules/nm2', 'molecules/unitcell', 'wt%', 'ml/g', 'mmol/g', 'mg/m2', 'mol/m3', 'kg/mol', 'molecules/formula unit', 'particles/nm^3', 'g/l', 'g/g', 'g/cm3', 'cm3(STP)/cm3', 'mol/l', 'molecules/cage', 'mol/g', 'molecules/8 unit cells', 'mmol/kg', 'cm3/m2', 'micromoles/m2'}


In [45]:
print(len(filtered_isotherms))
filtered_isotherms

5638


,DOI,filename,Material Name,Temperature (K),Equivalent CO2 Pressure (bar),Loading Estimate,Loading Units
0,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm13,Zn(BDC)(DMBPY) 0.5,288,0.000392,0.004396782798505759,wt%
1,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm14,Zn(BDC)(DMBPY) 0.5,298,0.000405,0.0015667002826212056,wt%
2,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm15,Zn(BDC)(DMBPY) 0.5,303,0.000412,0.0009340980703698181,wt%
3,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm16,Zn(BDC)(DMBPY) 0.5,308,0.000419,0.0006385104038903775,wt%
4,10.1002/adfm.201101479,10.1002adfm.201101479.Isotherm17,Zn(NDC)(DMBPY) 0.5,288,0.000392,0.0012085256090980118,wt%
...,...,...,...,...,...,...,...
5633,10.8888/s0167-2991(08)63564-8,10.8888s0167-2991(08)63564-8.Isotherm2,Carbon,303,0.000412,0.0017861356675803147,mmol/g
5634,10.8888/s10450-012-9407-1,10.8888s10450-012-9407-1.Isotherm1,ZIF-8,298,0.000405,0.03428099408019676,mmol/g
5635,10.8888/s10450-012-9407-1,10.8888s10450-012-9407-1.Isotherm11,ZIF-8,298,0.000405,0.00023533715518857045,mmol/g
5636,10.8888/s10450-012-9407-1,10.8888s10450-012-9407-1.Isotherm13,ZIF-8,308,0.000419,0.00014813996143904956,mmol/g


In [46]:
# Filter by loading characteristics
print(len(filtered_isotherms), 'Total Isotherms')

# Isotherms that use weight fraction
wt_frac_isotherms = filtered_isotherms[filtered_isotherms['Loading Units'] == 'wt%']
#display(wt_frac_isotherms)
print(len(wt_frac_isotherms), 'Isotherms use wt%')
wt_frac_isotherms.to_csv('co2_loading_wtfrac.csv')

# Work withIsotherms that use mmol/g
std_isotherms = filtered_isotherms[filtered_isotherms['Loading Units'] == 'mmol/g']
#display(std_isotherms)
#print(len(std_isotherms))
print(len(std_isotherms), 'Isotherms use mmol/g or were converted to mmol/g')

# Break into Subgroups
group1 = std_isotherms[(std_isotherms['Loading Estimate'] >= 0.5) & (std_isotherms['Loading Estimate'] < 1.0)]
group2 = std_isotherms[(std_isotherms['Loading Estimate'] >= 1.0) & (std_isotherms['Loading Estimate'] < 1.5)]
group3 = std_isotherms[std_isotherms['Loading Estimate'] >= 1.5]
group1.to_csv('co2_loading_0.5-1mmolpg.csv')
group2.to_csv('co2_loading_1-1.5mmolpg.csv')
group3.to_csv('co2_loading_gt1.5mmolpg.csv')

print(len(filtered_isotherms)-len(wt_frac_isotherms)-len(std_isotherms), 'Isotherms use some other units type')

5638 Total Isotherms
239 Isotherms use wt%
4393 Isotherms use mmol/g or were converted to mmol/g
1006 Isotherms use some other units type


In [10]:
# Boiler plate code to examine edge cases
# filename = '10.1007s10450-018-9958-x.Lab08'  # this is from the CO2 interlab study. shouldn't show negative
  #aha, this is a failure of linear interpolation.

# URL = 'https://adsorption.nist.gov/isodb/api/isotherm/'+filename+'.json&k=dontrackmeplease'
# isotherm_full = requests.get(URL).json()
# p = [0.]+[x['pressure'] for x in isotherm_full['isotherm_data']]
# q = [0.]+[x['species_data'][0]['adsorption'] for x in isotherm_full['isotherm_data']]

# print(p,q)

# T_iso = isotherm_full["temperature"]
# p_CO2_iso = p_CO2_atm * T_iso / T_atm  # equivalent pressure for isotherm temperature

# print(T_iso, p_CO2_iso)
